In [224]:
!pip3 install pandas scikit-learn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [225]:
# Load the data
# from google.colab import drive
# drive.mount('/content/drive')
file_path = '/content/drive/My Drive/england.csv'
file_path = 'england.csv'
df = pd.read_csv(file_path)
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,home_odds,draw_odds,away_odds,over25_odds,under25_odds
0,E0,11/08/2023,20:00,Burnley,Man City,0,3,A,0,2,A,C Pawson,8.00,5.5,1.33,1.67,2.20
1,E0,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,H,M Oliver,1.18,7.0,15.00,1.50,2.63
2,E0,12/08/2023,15:00,Bournemouth,West Ham,1,1,D,0,0,D,P Bankes,2.70,3.4,2.55,1.73,2.10
3,E0,12/08/2023,15:00,Brighton,Luton,4,1,H,1,0,H,D Coote,1.33,5.5,9.00,1.53,2.50
4,E0,12/08/2023,15:00,Everton,Fulham,0,1,A,0,0,D,S Attwell,2.20,3.4,3.30,2.10,1.73


In [226]:
# Data Preprocessing
# Convert categorical columns to numerical
df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y")
df['week_day'] = df['Date'].dt.weekday
# Function to replace ':' with '.' and convert to float
def convert_time_to_float(time_str):
    # Split the time string by ':'
    parts = time_str.split(':')
    # Convert hours and minutes to float
    hours = int(parts[0])
    minutes = int(parts[1]) / 60
    # Return the sum of hours and converted minutes as float
    return hours + minutes

# Apply the conversion function to the 'time' column
df['time_in_hours'] = df['Time'].apply(convert_time_to_float)


df['FTR'] = df['FTR'].map({'H': 1, 'D': 0, 'A': -1})
df['HTR'] = df['HTR'].map({'H': 1, 'D': 0, 'A': -1})

df.head(20)


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,home_odds,draw_odds,away_odds,over25_odds,under25_odds,week_day,time_in_hours
0,E0,2023-08-11,20:00,Burnley,Man City,0,3,-1,0,2,-1,C Pawson,8.00,5.50,1.33,1.67,2.20,4,20.00
1,E0,2023-08-12,12:30,Arsenal,Nott'm Forest,2,1,1,2,0,1,M Oliver,1.18,7.00,15.00,1.50,2.63,5,12.50
2,E0,2023-08-12,15:00,Bournemouth,West Ham,1,1,0,0,0,0,P Bankes,2.70,3.40,2.55,1.73,2.10,5,15.00
3,E0,2023-08-12,15:00,Brighton,Luton,4,1,1,1,0,1,D Coote,1.33,5.50,9.00,1.53,2.50,5,15.00
4,E0,2023-08-12,15:00,Everton,Fulham,0,1,-1,0,0,0,S Attwell,2.20,3.40,3.30,2.10,1.73,5,15.00
5,E0,2023-08-12,15:00,Sheffield United,Crystal Palace,0,1,-1,0,0,0,J Brooks,3.00,3.30,2.38,2.50,1.53,5,15.00
6,E0,2023-08-12,17:30,Newcastle,Aston Villa,5,1,1,2,1,1,A Madley,1.75,3.75,4.60,1.67,2.20,5,17.50
7,E0,2023-08-13,14:00,Brentford,Tottenham,2,2,0,2,2,0,R Jones,2.75,3.40,2.45,1.73,2.10,6,14.00
8,E0,2023-08-13,16:30,Chelsea,Liverpool,1,1,0,1,1,0,A Taylor,2.90,3.40,2.38,1.53,2.50,6,16.50
9,E0,2023-08-14,20:00,Man United,Wolves,1,0,1,0,0,0,S Hooper,1.33,5.50,9.00,1.62,2.30,0,20.00


In [227]:
#Encode home team and away team
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['HomeTeam'] = encoder.fit_transform(df['HomeTeam'])
df['AwayTeam'] = encoder.fit_transform(df['AwayTeam'])
df['Referee'] = encoder.fit_transform(df['Referee'])

df.head(20)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,home_odds,draw_odds,away_odds,over25_odds,under25_odds,week_day,time_in_hours
0,E0,2023-08-11,20:00,5,12,0,3,-1,0,2,-1,3,8.00,5.50,1.33,1.67,2.20,4,20.00
1,E0,2023-08-12,12:30,0,15,2,1,1,2,0,1,14,1.18,7.00,15.00,1.50,2.63,5,12.50
2,E0,2023-08-12,15:00,2,18,1,1,0,0,0,0,16,2.70,3.40,2.55,1.73,2.10,5,15.00
3,E0,2023-08-12,15:00,4,11,4,1,1,1,0,1,5,1.33,5.50,9.00,1.53,2.50,5,15.00
4,E0,2023-08-12,15:00,8,9,0,1,-1,0,0,0,22,2.20,3.40,3.30,2.10,1.73,5,15.00
5,E0,2023-08-12,15:00,16,7,0,1,-1,0,0,0,8,3.00,3.30,2.38,2.50,1.53,5,15.00
6,E0,2023-08-12,17:30,14,1,5,1,1,2,1,1,0,1.75,3.75,4.60,1.67,2.20,5,17.50
7,E0,2023-08-13,14:00,3,17,2,2,0,2,2,0,18,2.75,3.40,2.45,1.73,2.10,6,14.00
8,E0,2023-08-13,16:30,6,10,1,1,0,1,1,0,1,2.90,3.40,2.38,1.53,2.50,6,16.50
9,E0,2023-08-14,20:00,13,19,1,0,1,0,0,0,24,1.33,5.50,9.00,1.62,2.30,0,20.00


In [228]:
# Select features and target variable
# features = ['HomeTeam',	'AwayTeam', 'home_odds','Referee',	'draw_odds', 'away_odds', 'over25_odds', 'under25_odds','week_day', 'time_in_hours']
features = ['HomeTeam',	'AwayTeam', 'home_odds','Referee',	'draw_odds', 'away_odds', 'over25_odds', 'under25_odds','week_day', ]
# features = ['home_odds',	'draw_odds','Referee' ,'away_odds', 'over25_odds', 'under25_odds','week_day']


target = 'FTR'

X = df[features]
y = df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the model
model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [229]:

# Make predictions
y_pred = model.predict(X_test)

In [230]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Output the evaluation metrics
print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

Accuracy: 0.5087719298245614
Confusion Matrix:
[[22  1 11]
 [13  1 16]
 [11  4 35]]
Classification Report:
              precision    recall  f1-score   support

          -1       0.48      0.65      0.55        34
           0       0.17      0.03      0.06        30
           1       0.56      0.70      0.62        50

    accuracy                           0.51       114
   macro avg       0.40      0.46      0.41       114
weighted avg       0.43      0.51      0.45       114



In [231]:
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.columns = ['prediction']

y_pred_df = y_pred_df.reset_index()
y_pred_df.head()

,index,prediction
0,0,1
1,1,-1
2,2,1
3,3,-1
4,4,1


In [232]:
y_test_df = pd.DataFrame(y_test)
y_test_df.columns = ['result']
y_test_df = y_test_df.reset_index()
y_test_df.head()

,index,result
0,266,1
1,261,1
2,265,-1
3,39,-1
4,33,-1


In [233]:
X_test_df = X_test.reset_index()
X_test_df.shape
X_test_df.to_csv('test_data.csv', index=False)
y_test_df.to_csv('y_test.csv', index=False)
y_pred_df.to_csv('y_pred.csv', index=False)

In [234]:
# results_df = pd.concat([X_test,y_pred_df,y_test_df],axis =1)
result_odds = []
for row in range(114):
  if y_pred_df.prediction[row] == 1 and y_pred_df.prediction[row] == 1:
    result_odds.append(X_test_df.home_odds[row])
    print("home win sucesss")
  elif y_pred_df.prediction[row] == 0 and y_pred_df.prediction[row] == 0:
    result_odds.append(X_test_df.draw_odds[row])
    print("Draw sucesss")
  elif y_pred_df.prediction[row] == -1 and y_pred_df.prediction[row] == -1:
    result_odds.append(X_test_df.away_odds[row])
    print("Away win sucesss")
  else:
      result_odds.append(-1)
      print("wrong prediction")

print(result_odds)
print("results value = ",sum(result_odds))


home win sucesss
Away win sucesss
home win sucesss
Away win sucesss
home win sucesss
home win sucesss
home win sucesss
Away win sucesss
Away win sucesss
home win sucesss
Away win sucesss
home win sucesss
Away win sucesss
home win sucesss
home win sucesss
home win sucesss
Away win sucesss
home win sucesss
home win sucesss
Away win sucesss
home win sucesss
Away win sucesss
home win sucesss
Away win sucesss
home win sucesss
home win sucesss
home win sucesss
Away win sucesss
home win sucesss
Away win sucesss
home win sucesss
Away win sucesss
home win sucesss
Away win sucesss
Away win sucesss
Draw sucesss
home win sucesss
Away win sucesss
Away win sucesss
home win sucesss
Away win sucesss
home win sucesss
home win sucesss
home win sucesss
Away win sucesss
home win sucesss
home win sucesss
Away win sucesss
home win sucesss
home win sucesss
Draw sucesss
Away win sucesss
Away win sucesss
Away win sucesss
home win sucesss
Away win sucesss
Away win sucesss
home win sucesss
home win sucesss
home 

In [235]:
#PREDICT FOR ME
predi = model.predict()
print(predi)

TypeError: ForestClassifier.predict() missing 1 required positional argument: 'X'